### Part 1

Imports

In [1]:
! pip install --user pandas
! pip install --user numpy
import pandas as pd
import numpy as np

Fetch response from URL and convert to dataframe

In [2]:
import requests
from requests import get
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)
df = pd.read_html(response.text, header=0)[0]  # use first table on page

Cleaning and filtering

In [3]:
df.rename(columns={"Postcode": "PostalCode"}, inplace=True)
df.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)
is_assigned = df['Borough']!='Not assigned'
df = df[is_assigned].reset_index(drop=True)  # remove unassigned boroughs
df['Neighborhood'] = np.where(df.Neighborhood == 'Not assigned',df.Borough,df.Neighborhood)  # rename unassigned neighborhoods to borough name
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Group by postal code and join rows with same postal code (comma separated)

In [4]:
df = df.groupby(['PostalCode','Borough']).agg({'Neighborhood': lambda x: ','.join(x)}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df.shape

(103, 3)

### Part 2

Get the latitude and the longitude coordinates with geocoder

In [6]:
''' needs extremly long, so used csv in cell below
! pip install --user geocoder
import geocoder # import geocoder
long = []
lat = []
for val in df['PostalCode']:
    
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(val))
        lat_lng_coords = g.latlng
        
    lat.append(lat_lng_coords[0])
    long.append(lat_lng_coords[1])

df['Longitude'] = long
df['Latitude'] = lat
'''
print()

Get the latitude and the longitude coordinates from csv

In [7]:
df_g = pd.read_csv('https://cocl.us/Geospatial_data')
df_g.head()
df_merge = pd.merge(df, df_g, left_on='PostalCode', right_on='Postal Code', how='left')  # merge the frames on postal code
df = df_merge.drop('Postal Code', axis=1)  # remove duplicate postal code row
print(df.shape)
df.head()


(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3

Imports and configuring

In [8]:
from IPython.utils import io
with io.capture_output() as captured:
    ! pip install --user matplotlib sklearn folium geopy
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Libraries imported.')

Libraries imported.


Use geopy to fetch latitude and longitude for Toronto

In [9]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


create a map of Toronto with neighborhoods

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = df
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare credentials and config

In [11]:
CLIENT_ID = 'TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41'  #'E2DOPROLPU5FDTWOQTEM53X0HEVMCP3B4EL5T3IZ0KCW2GE5' # your Foursquare ID
CLIENT_SECRET = 'LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0'  #'EHCD1HMHZKIOJ5ZQIS3VGIDMOXJCRARFVQVYMHGW4EF01ZC1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

Function to fetch nearby venues

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        print(url)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Fetched %d venues' % len(nearby_venues))
    return(nearby_venues)

Create dataframe with venues for Toronto

In [13]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
toronto_venues.to_csv('toronto_venues.csv')

Rouge,Malvern
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100
Highland Creek,Rouge Hill,Port Union
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7845351,-79.16049709999999&radius=500&limit=100
Guildwood,Morningside,West Hill
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100
Woburn
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7709921,-79.2169174000000

Downsview Central
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7284964,-79.49569740000001&radius=500&limit=100
Downsview Northwest
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7616313,-79.52099940000001&radius=500&limit=100
Victoria Village
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=500&limit=100
Woodbine Gardens,Parkview Hill
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7063972,-79.309937&rad

The Annex,North Midtown,Yorkville
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.6727097,-79.40567840000001&radius=500&limit=100
Harbord,University of Toronto
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.6626956,-79.4000493&radius=500&limit=100
Chinatown,Grange Park,Kensington Market
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.6532057,-79.4000493&radius=500&limit=100
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&clien

Humber Summit
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7563033,-79.56596329999999&radius=500&limit=100
Emery,Humberlea
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.7247659,-79.53224240000002&radius=500&limit=100
Weston
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.706876,-79.51818840000001&radius=500&limit=100
Westmount
https://api.foursquare.com/v2/venues/explore?&client_id=TNGFOHDCJSFD2O1OURUBBF2DWUNDPEBTBZLJRCATGZCM2M41&client_secret=LXOE3BXPKGMYFMVYFBJVVDRVGDVSD1TL2NST3D4QQPXYCCC0&v=20180605&ll=43.696319,-79.53224240000002&radius=500&limit=100
Kingsview Village,Marti

In [14]:
# debug print all categories
print(toronto_venues.shape)
toronto_venues.head()
a = pd.DataFrame(toronto_venues['Venue Category'].unique().tolist())
a.sort_values(0)

(2235, 7)


,0
249,Accessories Store
163,Afghan Restaurant
112,Airport
241,Airport Food Court
243,Airport Gate
239,Airport Lounge
245,Airport Service
242,Airport Terminal
31,American Restaurant
172,Antique Shop


In [15]:
# check similarity of venues
!pip install --user python-Levenshtein
import itertools
import Levenshtein

venue_list = toronto_venues['Venue Category'].unique()
for pair in itertools.combinations(venue_list, 2):
    similarity = Levenshtein.jaro(pair[0], pair[1])
    if similarity >= 0.90:  # and similarity != 1.0:
        print(pair, similarity)


('Mexican Restaurant', 'American Restaurant') 0.9103313840155945
('Japanese Restaurant', 'Taiwanese Restaurant') 0.9149122807017545
('Cosmetics Shop', 'Comic Shop') 0.9047619047619048
('Hotel', 'Hostel') 0.9444444444444445


Inspect number of venues in descending order

In [16]:
toronto_venues['Neighborhood'].value_counts().to_frame()


,Neighborhood
"Commerce Court,Victoria Hotel",100
"First Canadian Place,Underground city",100
St. James Town,100
"Harbourfront East,Toronto Islands,Union Station",100
"Design Exchange,Toronto Dominion Centre",100
"Ryerson,Garden District",100
"Adelaide,King,Richmond",100
Stn A PO Boxes 25 The Esplanade,96
"Chinatown,Grange Park,Kensington Market",94
Central Bay Street,85


In [17]:
# Unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


#### One hot encoding for venue categories
As there is a category called 'Neighborhood' we cannot simply add column 'Neighborhood' from venues dataframe
to one hot encoded data frame, so rename the category column first

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# rename venue category column 'Neighborhood'
toronto_onehot.rename(columns={"Neighborhood": "Neighborhood Venue"}, inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot = toronto_onehot[ ['Neighborhood'] + [ col for col in toronto_onehot.columns if col != 'Neighborhood' ] ]

#toronto_onehot[toronto_onehot['Airport Food Court'] != 0].tail()
print(toronto_onehot.shape)
toronto_onehot.head(3)


(2235, 279)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,

In [19]:
# check correct shape (should be one more column than unique categories)
toronto_onehot.shape

(2235, 279)

Group by by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,

In [21]:
toronto_grouped.shape

(100, 279)

Function to return most common venues in sorted order

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create dataframe with top 10 venues for each neighborhood

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bakery,Asian Restaurant,Burger Joint,Bar,Sushi Restaurant
1,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Grocery Store,Sandwich Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Eastern European Restaurant,Dumpling Restaurant,Drugstore
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Gym,Airport Terminal,Deli / Bodega


#### Cluster Neighborhoods with k-means

In [29]:
# set number of clusters
kclusters = 5
toronto_clustered = toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustered)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
kmeans.labels_

array([1, 1, 3, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 3, 1, 1, 3, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 4, 0, 1, 3, 0, 3, 4, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 4, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 0, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 3, 0, 0, 0, 3, 1, 0, 0, 0, 1, 3], dtype=int32)

dataframe with cluster label and top 10 venues

In [31]:
# add clustering labels
df_n = neighborhoods_venues_sorted.copy()
df_n.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = df

#print(df_n.dtypes)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(df_n.set_index('Neighborhood'), on='Neighborhood')

# drop NaN columns (not all neighborhoods had venues)
toronto_merged = toronto_merged.dropna().reset_index(drop=True)

# convert float to int (join converted to float because of NaN values)
toronto_merged['Cluster Label'] = toronto_merged['Cluster Label'].astype(int)
#print(toronto_merged.dtypes)

toronto_merged.tail() # check the last columns!

Cluster Label              int32
Neighborhood              object
1st Most Common Venue     object
2nd Most Common Venue     object
3rd Most Common Venue     object
4th Most Common Venue     object
5th Most Common Venue     object
6th Most Common Venue     object
7th Most Common Venue     object
8th Most Common Venue     object
9th Most Common Venue     object
10th Most Common Venue    object
dtype: object
PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Label               int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,M9N,York,Weston,43.706876,-79.518188,3,Park,Convenience Store,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
97,M9P,Etobicoke,Westmount,43.696319,-79.532242,0,Pizza Place,Middle Eastern Restaurant,Intersection,Discount Store,Coffee Shop,Sandwich Place,Chinese Restaurant,Yoga Studio,Diner,Dog Run
98,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,1,Park,Pizza Place,Mobile Phone Shop,Bus Line,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
99,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437,0,Pizza Place,Grocery Store,Sandwich Place,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Eastern European Restaurant,Dumpling Restaurant,Drugstore
100,M9W,Etobicoke,Northwest,43.706748,-79.594054,1,Rental Car Location,Bar,Drugstore,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


Create a map with the culsters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

In [35]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,Scarborough,0,Coffee Shop,Korean Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,0,Convenience Store,Discount Store,Coffee Shop,Hobby Shop,Department Store,Drugstore,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
13,Scarborough,0,Pharmacy,Pizza Place,Fried Chicken Joint,Noodle House,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Convenience Store,Bank,Fast Food Restaurant
15,Scarborough,0,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Pizza Place,Breakfast Spot,Sandwich Place,Coffee Shop,Indian Restaurant,Pharmacy,Gym
16,North York,0,Pool,Fast Food Restaurant,Golf Course,Mediterranean Restaurant,Dog Run,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
22,North York,0,Butcher,Pharmacy,Pizza Place,Coffee Shop,Discount Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
25,North York,0,Gym,Asian Restaurant,Beer Store,Coffee Shop,Fast Food Restaurant,Bike Shop,Sandwich Place,Sporting Goods Shop,Japanese Restaurant,Italian Restaurant
26,North York,0,Coffee Shop,Bridal Shop,Pharmacy,Deli / Bodega,Supermarket,Bank,Sushi Restaurant,Restaurant,Fast Food Restaurant,Diner
31,North York,0,Grocery Store,Liquor Store,Athletics & Sports,Discount Store,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,Scarborough,1,Rental Car Location,Mexican Restaurant,Electronics Store,Spa,Breakfast Spot,Pizza Place,Medical Center,Intersection,Dim Sum Restaurant,Diner
4,Scarborough,1,Hakka Restaurant,Fried Chicken Joint,Bank,Athletics & Sports,Caribbean Restaurant,Thai Restaurant,Gas Station,Bakery,Drugstore,Donut Shop
5,Scarborough,1,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
7,Scarborough,1,Bakery,Bus Line,Soccer Field,Bus Station,Metro Station,Fast Food Restaurant,Intersection,Park,General Travel,General Entertainment
8,Scarborough,1,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
9,Scarborough,1,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Pet Store,Vietnamese Restaurant,Brewery,Chinese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
11,Scarborough,1,Middle Eastern Restaurant,Sandwich Place,Auto Garage,Breakfast Spot,Shopping Mall,Bakery,Yoga Studio,Discount Store,Dog Run,Doner Restaurant
12,Scarborough,1,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [37]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,Etobicoke,2,Filipino Restaurant,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [38]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3,Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
21,North York,3,Park,Bank,Convenience Store,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
23,North York,3,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
28,North York,3,Airport,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
38,East York,3,Park,Convenience Store,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
48,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
72,York,3,Park,Women's Store,Market,Fast Food Restaurant,Comfort Food Restaurant,Dim Sum Restaurant,Event Space,Colombian Restaurant,Ethiopian Restaurant,Empanada Restaurant
77,North York,3,Bakery,Park,Construction & Landscaping,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
88,Etobicoke,3,Pool,Park,River,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dance Studio
96,York,3,Park,Convenience Store,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [39]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,North York,4,Food Truck,Baseball Field,Yoga Studio,Fast Food Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
89,Etobicoke,4,Deli / Bodega,Baseball Field,Yoga Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
95,North York,4,Baseball Field,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
